In [61]:
from google.cloud import bigquery
import pandas
import requests
import json
import os

pandas.set_option('display.max_rows', None)
pandas.set_option('display.max_columns', None)

In [ ]:
# import requests
# import pandas as pd

# def get_acs_data(api_key, year, table_id, state_fips):
#     # Base URL for the Census API
#     base_url = f"https://api.census.gov/data/{year}/acs/acs5"
    
#     # Constructing the API request
#     params = {
#         "get": f"NAME,{table_id}_001E",
#         "for": "county:*",
#         "in": f"state:{state_fips}",
#         "key": api_key
#     }
    
#     # Making the API request
#     response = requests.get(base_url, params=params)
    
#     # Check if the request was successful
#     if response.status_code == 200:
#         # Convert the response to a pandas DataFrame
#         data = pd.DataFrame(response.json()[1:], columns=response.json()[0])
#         return data
#     else:
#         print(f"Error: {response.status_code}")
#         return None

# # Example usage
# api_key = "YOUR_CENSUS_API_KEY"
# year = "2021"
# table_id = "B01003"  # Total population
# state_fips = "06"  # California

# data = get_acs_data(api_key, year, table_id, state_fips)

# if data is not None:
#     print(data.head())

In [62]:
api_key = os.getenv('CENSUS_API_KEY')
#reference: https://www.census.gov/data/developers/guidance/api-user-guide/example-api-queries.html

state = '40'
table = 'B18101I_006E' # Estimate!!Total:!!18 to 64 years:!!With a disability
url = f'https://api.census.gov/data/2022/acs/acs1?get=NAME,{table}&for=county:*&in=state:{state}&key={api_key}'

response = requests.get(url)
        
file = json.loads(response.text)

df = pandas.DataFrame(file)

df

,0,1,2,3
0,NAME,B18101I_006E,state,county
1,"Canadian County, Oklahoma",733,40,017
2,"Cleveland County, Oklahoma",2231,40,027
3,"Comanche County, Oklahoma",1519,40,031
4,"Creek County, Oklahoma",346,40,037
5,"Muskogee County, Oklahoma",None,40,101
6,"Oklahoma County, Oklahoma",6768,40,109
7,"Payne County, Oklahoma",145,40,119
8,"Pottawatomie County, Oklahoma",367,40,125
9,"Rogers County, Oklahoma",308,40,131


In [63]:
df = df.drop(columns=2)

df

,0,1,3
0,NAME,B18101I_006E,county
1,"Canadian County, Oklahoma",733,017
2,"Cleveland County, Oklahoma",2231,027
3,"Comanche County, Oklahoma",1519,031
4,"Creek County, Oklahoma",346,037
5,"Muskogee County, Oklahoma",None,101
6,"Oklahoma County, Oklahoma",6768,109
7,"Payne County, Oklahoma",145,119
8,"Pottawatomie County, Oklahoma",367,125
9,"Rogers County, Oklahoma",308,131


In [64]:
df[['column1','column2','column3']] = df[0].str.split(' ', expand=True)

cleaned_df

,18_69_disabled,county_fip,county
1,733,017,CANADIAN
2,2231,027,CLEVELAND
3,1519,031,COMANCHE
4,346,037,CREEK
5,None,101,MUSKOGEE
6,6768,109,OKLAHOMA
7,145,119,PAYNE
8,367,125,POTTAWATOMIE
9,308,131,ROGERS
10,5564,143,TULSA


In [65]:
cleaned_df = df.drop([0,'column2', 'column3'], axis=1)

cleaned_df

,1,3,column1
0,B18101I_006E,county,NAME
1,733,017,Canadian
2,2231,027,Cleveland
3,1519,031,Comanche
4,346,037,Creek
5,None,101,Muskogee
6,6768,109,Oklahoma
7,145,119,Payne
8,367,125,Pottawatomie
9,308,131,Rogers


In [66]:
cleaned_df = cleaned_df.rename(columns ={
    1:'disabled_18to69',
    3:'county_fip',
    'column1':'county'
})

cleaned_df

,disabled_18to69,county_fip,county
0,B18101I_006E,county,NAME
1,733,017,Canadian
2,2231,027,Cleveland
3,1519,031,Comanche
4,346,037,Creek
5,None,101,Muskogee
6,6768,109,Oklahoma
7,145,119,Payne
8,367,125,Pottawatomie
9,308,131,Rogers


In [67]:
cleaned_df = cleaned_df.drop(0)

cleaned_df

,disabled_18to69,county_fip,county
1,733,017,Canadian
2,2231,027,Cleveland
3,1519,031,Comanche
4,346,037,Creek
5,None,101,Muskogee
6,6768,109,Oklahoma
7,145,119,Payne
8,367,125,Pottawatomie
9,308,131,Rogers
10,5564,143,Tulsa


In [68]:
cleaned_df['county'] = cleaned_df['county'].str.upper()

cleaned_df

,disabled_18to69,county_fip,county
1,733,017,CANADIAN
2,2231,027,CLEVELAND
3,1519,031,COMANCHE
4,346,037,CREEK
5,None,101,MUSKOGEE
6,6768,109,OKLAHOMA
7,145,119,PAYNE
8,367,125,POTTAWATOMIE
9,308,131,ROGERS
10,5564,143,TULSA


In [69]:
cleaned_df.to_csv('/workspaces/bq-public/bq_dbt/seeds/census_seed.csv')